<a href="https://colab.research.google.com/github/oakthyago/NLP_study_one/blob/main/NLP_Introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using Kaggle API to import the dataset saving drive space

In [1]:
from google.colab import files
import os
import shutil

# Upload kaggle.json
uploaded = files.upload()

# Move kaggle.json to the .kaggle directory
os.makedirs('/root/.kaggle', exist_ok=True)
shutil.move('kaggle.json', '/root/.kaggle/kaggle.json')

# Set permissions
!chmod 600 /root/.kaggle/kaggle.json

# Verify setup
!kaggle datasets list


Saving kaggle.json to kaggle.json
ref                                                              title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
lainguyn123/student-performance-factors                          Student Performance Factors                          94KB  2024-09-02 10:53:57           5904        126  1.0              
hanaksoy/customer-purchasing-behaviors                           Customer Purchasing Behaviors                         1KB  2024-09-01 22:18:07           3539         52  1.0              
haseebindata/student-performance-predictions                     Student Performance Predictions                       9KB  2024-08-17 06:57:57          11594        247  0.9411765        
computingvictor/2024-

In [2]:
# prompt: kaggle datasets download -d luisfredgs/imdb-ptbr and put on data =

!kaggle datasets download -d luisfredgs/imdb-ptbr
!unzip imdb-ptbr.zip



Dataset URL: https://www.kaggle.com/datasets/luisfredgs/imdb-ptbr
License(s): unknown
 91% 44.0M/48.4M [00:00<00:00, 52.8MB/s]
100% 48.4M/48.4M [00:00<00:00, 55.0MB/s]
Archive:  imdb-ptbr.zip
  inflating: imdb-reviews-pt-br.csv  


In [3]:
import pandas as pd


# Load the dataset into the 'data' variable (assuming the file is a CSV, adjust the filename if necessary)
reviews = pd.read_csv('/content/imdb-reviews-pt-br.csv')



In [4]:
reviews.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


In [5]:
reviews.sentiment.value_counts()

,count
sentiment,
neg,24765
pos,24694


In [6]:
# prompt: reviews.sentiment to bololin value 0 neg 1 pos

reviews['bool_sentiment'] = reviews['sentiment'].apply(lambda x: 0 if x == 'neg' else 1)
reviews.bool_sentiment.value_counts()

,count
bool_sentiment,
0,24765
1,24694


In [7]:
reviews.tail()

,id,text_en,text_pt,sentiment,bool_sentiment
49454,49456,"Seeing as the vote average was pretty low, and...","Como a média de votos era muito baixa, e o fat...",pos,1
49455,49457,"The plot had some wretched, unbelievable twist...",O enredo teve algumas reviravoltas infelizes e...,pos,1
49456,49458,I am amazed at how this movieand most others h...,Estou espantado com a forma como este filme e ...,pos,1
49457,49459,A Christmas Together actually came before my t...,A Christmas Together realmente veio antes do m...,pos,1
49458,49460,Working-class romantic drama from director Mar...,O drama romântico da classe trabalhadora do di...,pos,1


In [8]:
# prompt: from scikitlearn import test train split




Bag of words and Classification

In [29]:
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import train_test_split



bag_of_words = CountVectorizer(lowercase=False, max_features=100).fit_transform(reviews['text_pt'])

X_train, X_test, y_train, y_test = train_test_split(
    bag_of_words,
    reviews['bool_sentiment'],
    test_size=0.2,
    random_state=42
)


In [30]:
# prompt: bag_of_words SIZE

print(bag_of_words.shape)


(49459, 100)


In [34]:
# prompt: Log_Reg = logistic regretion.fit( X_train, y_train

from sklearn.linear_model import LogisticRegression

Log_Reg = LogisticRegression(max_iter=1000)
Log_Reg.fit(X_train, y_train)



LogisticRegression(max_iter=1000)

In [35]:
Log_Reg.score(X_test, y_test)

0.7019813991103923

In [38]:
# prompt: make the function text_classification that will get  reviews, text_pt and bool_sentiment

def text_classification(reviews, text_pt, bool_sentiment,N_Features):
  """
  This function performs text classification on movie reviews.

  Args:
    reviews: A pandas DataFrame containing the movie reviews.
    text_pt: The column name containing the review text.
    bool_sentiment: The column name containing the sentiment labels (0 or 1).

  Returns:
    The accuracy score of the Logistic Regression model.
  """
  bag_of_words = CountVectorizer(lowercase=False, max_features=N_Features).fit_transform(reviews[text_pt])
  X_train, X_test, y_train, y_test = train_test_split(
      bag_of_words,
      reviews[bool_sentiment],
      test_size=0.2,
      random_state=42
  )
  Log_Reg = LogisticRegression(max_iter=1000)
  Log_Reg.fit(X_train, y_train)
  return Log_Reg.score(X_test, y_test)


In [41]:
text_classification(reviews,'text_pt','bool_sentiment',100)

0.7019813991103923

In [42]:
text_classification(reviews,'text_pt','bool_sentiment',1000)

0.848160129397493

In [43]:
text_classification(reviews,'text_pt','bool_sentiment',5000)

0.8661544682571776